In [2]:
import random
import math
import time
import threading
import pygame
import pandas as pd
import sys
import os

In [3]:
#Path Setting
data_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx'
background = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\four_way_intersection.jpg'
car_up = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\up.png'
car_down = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Down.png'
car_left = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Left.png'
car_right = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Right.png'
signal_r = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\red.png'
signal_g = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\green.png'
signal_y = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\yellow.png'

In [4]:
#Load Image
back = pygame.image.load(background)
Cup = pygame.image.load(car_up)
Cdown = pygame.image.load(car_down)
Cleft = pygame.image.load(car_left)
Cright = pygame.image.load(car_right)
red = pygame.image.load(signal_r)
green = pygame.image.load(signal_g)
yellow = pygame.image.load(signal_y)

In [5]:
# Car sizes
carUp = Cup.get_rect().size
print(carUp)
carDown = Cdown.get_rect().size
carLeft = Cleft.get_rect().size
carRight = Cright.get_rect().size
print(carRight)

(40, 79)
(79, 40)


In [6]:
# Traffic signal coordinates
signalCoods = [(161, 106), (404, 106), (161, 404), (404, 404)]
# Stop line coordinates
stopLineCo = [[(224, 194), (294, 194)], [(404, 224), (404, 294)], [(304, 404), (374, 404)], [(194, 304), (194, 374)]]

In [7]:
# Read Excel file into a pandas DataFrame
try:
    data = pd.read_excel(data_path)
    # Display the DataFrame to see the imported data
    # print(data)  # Display the first few rows of the DataFrame
except FileNotFoundError:
    print("Excel file not found.")
except Exception as e:
    print("An error occurred:", e)

In [8]:
# Car initial positions
carCo_down = (224, 0)
carCo_up = (340, 521)
carCo_left = (521, 224)
carCo_right = (0, 340)
car_speed = 2.25  # Speed of the car (pixels per frame)
moving = True

In [142]:
# Car initial positions
carCo_down = (224, 0)
carCo_up = (340, 521)
carCo_left = (521, 224)
carCo_right = (0, 340)

# Car speeds
car_speed = 2.25  # Speed of the car (pixels per frame)

# Initialize pygame
pygame.init()
screenW = 600
screenH = 600
screen = pygame.display.set_mode((screenW, screenH))

# Set up initial positions and rects for cars
car_x_down, car_y_down = carCo_down
car_x_up, car_y_up = carCo_up
car_x_left, car_y_left = carCo_left
car_x_right, car_y_right = carCo_right

# Set up flags for car movements
moving_down = True
moving_up = True
moving_left = True
moving_right = True

# Create rects for cars
car_rect_down = pygame.Rect(car_x_down, car_y_down, Cdown.get_width(), Cdown.get_height())
car_rect_up = pygame.Rect(car_x_up, car_y_up, Cup.get_width(), Cup.get_height())
car_rect_left = pygame.Rect(car_x_left, car_y_left, Cleft.get_width(), Cleft.get_height())
car_rect_right = pygame.Rect(car_x_right, car_y_right, Cright.get_width(), Cright.get_height())

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False

    # Clear the screen
    screen.fill((255, 255, 255))

    # Blit the background image onto the screen
    screen.blit(back, (0, 0))

    # Blit the traffic signals onto the screen
    for signalCoord in signalCoods:
        screen.blit(red, signalCoord)

    # Move the cars horizontally if the flags are True
    if moving_down:
        car_y_down += car_speed
    if moving_up:
        car_y_up -= car_speed
    if moving_left:
        car_x_left -= car_speed
    if moving_right:
        car_x_right += car_speed

    # Ensure the cars stay within the screen bounds
    if car_x_down > screenW:
        car_x_down = 0
    if car_x_up < -carUp[0]:
        car_x_up = screenW
    if car_y_left < -carLeft[1]:
        car_y_left = screenH
    if car_y_right > screenH:
        car_y_right = 0

    # Check for collision with stop lines for each car
    for i, line_rect in enumerate(stopLineCo):
        pygame.draw.line(screen, (255, 0, 0), line_rect[0], line_rect[1], 3)
        
        if i == 0 and car_rect_down.colliderect(pygame.Rect(line_rect[0][0], line_rect[0][1], 4, 1)):
            moving_down = False
        if i == 1 and car_rect_left.colliderect(pygame.Rect(line_rect[0][0], line_rect[0][1], 1, 4)):
            moving_left = False
        if i == 2 and car_rect_up.colliderect(pygame.Rect(line_rect[1][0], line_rect[0][1], 4, 1)):
            moving_up = False
        if i == 3 and car_rect_right.colliderect(pygame.Rect(line_rect[0][0] - 2, line_rect[0][1], 4, line_rect[1][1] - line_rect[0][1])):
            moving_right = False

    # Update car rects positions
    car_rect_down.topleft = (car_x_down, car_y_down)
    car_rect_up.topleft = (car_x_up, car_y_up)
    car_rect_left.topleft = (car_x_left, car_y_left)
    car_rect_right.topleft = (car_x_right, car_y_right)

    # Blit the cars onto the screen at their updated positions
    screen.blit(Cdown, (car_x_down, car_y_down))
    screen.blit(Cup, (car_x_up, car_y_up))
    screen.blit(Cleft, (car_x_left, car_y_left))
    screen.blit(Cright, (car_x_right, car_y_right))

    # Update the display
    pygame.display.flip()

    # Add a small delay to control the speed of the cars
    pygame.time.delay(10)

pygame.quit()
sys.exit()


SystemExit: 

In [141]:
pygame.init()
screenW = 600
screenH = 600
screen = pygame.display.set_mode((screenW, screenH))
# Define the number of cars in each direction
num_cars_down = 2
num_cars_up = 2
num_cars_left = 2
num_cars_right = 2

carCo_down = (224, 0)
carCo_up = (340, 521)
carCo_left = (521, 224)
carCo_right = (0, 340)
# Create lists to hold the positions and rectangles of each car
car_positions_down = [(224, 0 - i * carDown[1] * 1.5) for i in range(num_cars_down)]
car_positions_up = [(340, 521 - i * carUp[1]+1 * 1.5) for i in range(num_cars_up)]
car_positions_left = [(521 - i * carLeft[0]+1 * 1.5, 224) for i in range(num_cars_left)]
car_positions_right = [(0 + i * carRight[0]+1 * 1.5, 340) for i in range(num_cars_right)]

car_rects_down = [pygame.Rect(pos[0], pos[1], carDown[0], carDown[1]) for pos in car_positions_down]
car_rects_up = [pygame.Rect(pos[0], pos[1], carUp[0], carUp[1]) for pos in car_positions_up]
car_rects_left = [pygame.Rect(pos[0], pos[1], carLeft[0], carLeft[1]) for pos in car_positions_left]
car_rects_right = [pygame.Rect(pos[0], pos[1], carRight[0], carRight[1]) for pos in car_positions_right]

# Set up flags for car movements
moving_down = [True] * num_cars_down
moving_up = [True] * num_cars_up
moving_left = [True] * num_cars_left
moving_right = [True] * num_cars_right

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False

    # Clear the screen
    screen.fill((255, 255, 255))

    # Blit the background image onto the screen
    screen.blit(back, (0, 0))

    # Blit the traffic signals onto the screen
    for signalCoord in signalCoods:
        screen.blit(red, signalCoord)

    # Move the cars horizontally if the flags are True
    for i, (car_rect, moving) in enumerate(zip(car_rects_down, moving_down)):
        if moving:
            car_rect.y += car_speed
        # Check for collision with other cars
        for other_rect in car_rects_down[:i] + car_rects_down[i + 1:]:
            if car_rect.colliderect(other_rect):
                moving_down[i] = False
                break
    # Check for collision with stop lines for each car
    for i, line_rect in enumerate(stopLineCo):
        pygame.draw.line(screen, (255, 0, 0), line_rect[0], line_rect[1], 3)
        # Check for collision with stop lines for each car
        for car_rect, moving in zip(car_rects_down, moving_down):
            if moving and car_rect.colliderect(pygame.Rect(line_rect[0][0], line_rect[0][1], 4, 1)):
                moving_down[car_rects_down.index(car_rect)] = False
                break
    
        # Similar collision detection for other directions...
    # Blit the cars onto the screen at their updated positions
    for car_rect in car_rects_down:
        screen.blit(Cdown, car_rect.topleft)
    for car_rect in car_rects_up:
        screen.blit(Cup, car_rect.topleft)
    for car_rect in car_rects_left:
        screen.blit(Cleft, car_rect.topleft)
    for car_rect in car_rects_right:
        screen.blit(Cright, car_rect.topleft)

    # Update the display
    pygame.display.flip()

    # Add a small delay to control the speed of the cars
    pygame.time.delay(10)

pygame.quit()
sys.exit()


SystemExit: 

In [111]:
import pygame
import sys
import pandas as pd

# Function to read timer values from Excel file
def read_timers_from_excel(data_path):
    try:
        data = pd.read_excel(data_path)
        timer_1 = data["Predicted_Time_1"].tolist()  # Extract timer 1 values
        timer_2 = data["Predicted_Time_2"].tolist()  # Extract timer 2 values
        timer_3 = data["Predicted_Time_3"].tolist()  # Extract timer 3 values
        timer_4 = data["Predicted_Time_4"].tolist()  # Extract timer 4 values
        return timer_1, timer_2, timer_3, timer_4
    except FileNotFoundError:
        print("Excel file not found.")
    except Exception as e:
        print("An error occurred:", e)
    return [], [], [], []

# Initialize pygame
pygame.init()

# Set the screen dimensions
screenW = 600
screenH = 600
screen = pygame.display.set_mode((screenW, screenH))

# Load car image
car_image = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Down.png')  # Replace "car_down.png" with the actual path to your car image

# Load traffic signal images
# red_signal = pygame.image.load("red_signal.png")  # Replace "red_signal.png" with the path to the red signal image
# green_signal = pygame.image.load("green_signal.png")  # Replace "green_signal.png" with the path to the green signal image

# Traffic signal coordinates
signalCoords = [(161, 106), (404, 106), (161, 404), (404, 404)]

# Set up initial position for the car
car_x = 300
car_y = 300

# Create a font for text rendering
font = pygame.font.Font(None, 24)

# Set up initial value for the timers
data_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx'  # Path to your Excel file
timer_1, timer_2, timer_3, timer_4 = read_timers_from_excel(data_path)

# Main loop
running = True
current_timer = 0  # Index of the current timer being active
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    screen.fill((255, 255, 255))

    # Blit the car onto the screen
    screen.blit(car_image, (car_x, car_y))
    screen.blit(back, (0,0))
    # Update and render timers
    timers = [timer_1, timer_2, timer_3, timer_4]
    for i, timer_list in enumerate(timers):
        if timer_list:  # Check if the timer list is not empty
            timer = timer_list[i]  # Get the current timer value
            if i == current_timer:  # Check if this is the current active timer
                timer_text = font.render("Timer " + str(i + 1) + ": " + str(round(timer, 2)), True, (255, 0, 0))
            else:
                timer_text = font.render("Timer " + str(i + 1) + ": Paused", True, (255, 0, 0))
            if i % 2 == 0:  # Top corners
                screen.blit(timer_text, (20, 20 + i // 2 * (screenH - 50)))
            else:  # Bottom corners
                screen.blit(timer_text, (screenW - 200, 20 + (i // 2) * (screenH - 50)))

            # Decrement timer value if it is the current active timer
            if timer > 0 and i == current_timer:
                timer_list[i] -= 0.01

            # Check if the current timer has reached 0 and update the index of the current active timer
            if timer <= 0 and i == current_timer:
                current_timer = (current_timer + 1) % 4

            # Render traffic signals
            if i == current_timer:
                screen.blit(green, signalCoords[i])
            else:
                screen.blit(red, signalCoords[i])

    # Update the display
    pygame.display.flip()

    # Add a small delay to control the frame rate
    pygame.time.delay(10)
    

pygame.quit()
sys.exit()


SystemExit: 

c:\Users\Kobiz\anaconda3\envs\AIproject\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [139]:
import pygame
import sys
import pandas as pd

# Function to read timer values from Excel file
def read_timers_from_excel(data_path):
    try:
        data = pd.read_excel(data_path)
        timer_1 = data["Predicted_Time_1"].tolist()  # Extract timer 1 values
        timer_2 = data["Predicted_Time_2"].tolist()  # Extract timer 2 values
        timer_3 = data["Predicted_Time_3"].tolist()  # Extract timer 3 values
        timer_4 = data["Predicted_Time_4"].tolist()  # Extract timer 4 values
        return timer_1, timer_2, timer_3, timer_4
    except FileNotFoundError:
        print("Excel file not found.")
    except Exception as e:
        print("An error occurred:", e)
    return [], [], [], []

# Initialize pygame
pygame.init()

# Set the screen dimensions
screenW = 600
screenH = 600
screen = pygame.display.set_mode((screenW, screenH))

# Load car image
car_image = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Down.png')  # Replace "car_down.png" with the actual path to your car image

# Load traffic signal images
red_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\red.png')  # Replace "red_signal.png" with the path to the red signal image
green_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\green.png')  # Replace "green_signal.png" with the path to the green signal image
yellow_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\yellow.png')  # Replace "yellow_signal.png" with the path to the yellow signal image

# Traffic signal coordinates
signalCoords = [(161, 106), (404, 106), (161, 404), (404, 404)]

# Set up initial position for the car
car_x = 300
car_y = 300

# Create a font for text rendering
font = pygame.font.Font(None, 24)
data_iterator = data.iterrows()
# Set up initial value for the timers
data_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx'  # Path to your Excel file
timer_1, timer_2, timer_3, timer_4 = read_timers_from_excel(data_path)
for index, row in data_iterator:  # Iterate over DataFrame rows

    # Main loop
    running = True
    current_timer = index  # Index of the current timer being active
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Clear the screen
        screen.fill((255, 255, 255))

        # Blit the car onto the screen
        screen.blit(car_image, (car_x, car_y))
        screen.blit(back, (0,0))
        # Update and render timers
        timers = [timer_1, timer_2, timer_3, timer_4]
        for i, timer_list in enumerate(timers):
            if timer_list:  # Check if the timer list is not empty
                timer = timer_list[i]  # Get the current timer value
                if i == current_timer:  # Check if this is the current active timer
                    timer_text = font.render("Timer " + str(i + 1) + ": " + str(round(timer, 2)), True, (255, 0, 0))
                else:
                    timer_text = font.render("Timer " + str(i + 1) + ": Paused", True, (255, 0, 0))
                if i % 2 == 0:  # Top corners
                    screen.blit(timer_text, (20, 20 + i // 2 * (screenH - 50)))
                else:  # Bottom corners
                    screen.blit(timer_text, (screenW - 200, 20 + (i // 2) * (screenH - 50)))

                # Decrement timer value if it is the current active timer
                if timer > 0 and i == current_timer:
                    timer_list[i] -= 0.01

                # Check if the current timer has reached 0 and update the index of the current active timer
                if timer <= 0 and i == current_timer:
                    current_timer = (current_timer + 1) % 4

                # Render traffic signals
                if i == current_timer:
                    if timer > 3:  # More than 3 seconds left, display green signal
                        screen.blit(green_signal, signalCoords[i])
                    elif timer > 1:  # More than 1 second left, display yellow signal
                        screen.blit(yellow_signal, signalCoords[i])
                    else:  # Less than or equal to 1 second left, display red signal
                        screen.blit(red_signal, signalCoords[i])
                else:
                    screen.blit(red_signal, signalCoords[i])  # Display red signal for inactive lanes

        # Update the display
        pygame.display.flip()

        # Add a small delay to control the frame rate
        pygame.time.delay(10)

    pygame.quit()
    sys.exit()


KeyboardInterrupt: 

In [136]:
import pygame
import sys
import pandas as pd

# Function to read timer values from Excel file
def read_timers_from_excel(data_path):
    try:
        data = pd.read_excel(data_path)
        timer_1 = data["Predicted_Time_1"].tolist()  # Extract timer 1 values
        timer_2 = data["Predicted_Time_2"].tolist()  # Extract timer 2 values
        timer_3 = data["Predicted_Time_3"].tolist()  # Extract timer 3 values
        timer_4 = data["Predicted_Time_4"].tolist()  # Extract timer 4 values
        return timer_1, timer_2, timer_3, timer_4
    except FileNotFoundError:
        print("Excel file not found.")
    except Exception as e:
        print("An error occurred:", e)
    return [], [], [], []

# Set the screen dimensions
screenW = 600
screenH = 600

# Set up initial value for the timers
data_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx'  # Path to your Excel file

while True:
    # Initialize pygame
    pygame.init()
    screen = pygame.display.set_mode((screenW, screenH))

    # Load car image
    car_image = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Down.png')  # Replace "car_down.png" with the actual path to your car image

    # Load traffic signal images
    red_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\red.png')  # Replace "red_signal.png" with the path to the red signal image
    green_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\green.png')  # Replace "green_signal.png" with the path to the green signal image
    yellow_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\yellow.png')  # Replace "yellow_signal.png" with the path to the yellow signal image

    # Traffic signal coordinates
    signalCoords = [(161, 106), (404, 106), (161, 404), (404, 404)]

    # Create a font for text rendering
    font = pygame.font.Font(None, 24)
    
    # Read timer values from Excel
    timer_1, timer_2, timer_3, timer_4 = read_timers_from_excel(data_path)

    for index in range(len(timer_1)):  # Iterate over rows of timer values
        # Set up initial position for the car
        car_x = 300
        car_y = 300

        # Main loop
        running = True
        current_timer = 0  # Index of the current timer being active
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            # Clear the screen
            screen.fill((255, 255, 255))

            # Blit the car onto the screen
            screen.blit(car_image, (car_x, car_y))

            # Update and render timers
            timers = [timer_1[index], timer_2[index], timer_3[index], timer_4[index]]
            for i, timer in enumerate(timers):
                if timer is not None:  # Check if the timer is not None (i.e., it's not missing)
                    if i == current_timer:  # Check if this is the current active timer
                        timer_text = font.render("Timer " + str(i + 1) + ": " + str(round(timer, 2)), True, (255, 0, 0))
                    else:
                        timer_text = font.render("Timer " + str(i + 1) + ": Paused", True, (255, 0, 0))
                    if i % 2 == 0:  # Top corners
                        screen.blit(timer_text, (20, 20 + i // 2 * (screenH - 50)))
                    else:  # Bottom corners
                        screen.blit(timer_text, (screenW - 200, 20 + (i // 2) * (screenH - 50)))

                    # Decrement timer value if it is the current active timer
                    if timer > 0 and i == current_timer:
                        timers[i] -= 0.01

                    # Check if the current timer has reached 0 and update the index of the current active timer
                    if timer <= 0 and i == current_timer:
                        current_timer = (current_timer + 1) % 4

                    # Render traffic signals
                    if i == current_timer:
                        if timer > 3:  # More than 3 seconds left, display green signal
                            screen.blit(green_signal, signalCoords[i])
                        elif timer > 1:  # More than 1 second left, display yellow signal
                            screen.blit(yellow_signal, signalCoords[i])
                        else:  # Less than or equal to 1 second left, display red signal
                            screen.blit(red_signal, signalCoords[i])
                    else:
                        screen.blit(red_signal, signalCoords[i])  # Display red signal for inactive lanes

            # Update the display
            pygame.display.flip()

            # Add a small delay to control the frame rate
            pygame.time.delay(10)

        pygame.quit()
    sys.exit()


error: video system not initialized

In [135]:
import pygame
import sys
import pandas as pd

# Function to read timer values from Excel file
def read_timers_from_excel(data_path):
    try:
        data = pd.read_excel(data_path)
        timer_1 = data["Predicted_Time_1"].tolist()  # Extract timer 1 values
        timer_2 = data["Predicted_Time_2"].tolist()  # Extract timer 2 values
        timer_3 = data["Predicted_Time_3"].tolist()  # Extract timer 3 values
        timer_4 = data["Predicted_Time_4"].tolist()  # Extract timer 4 values
        return timer_1, timer_2, timer_3, timer_4
    except FileNotFoundError:
        print("Excel file not found.")
    except Exception as e:
        print("An error occurred:", e)
    return [], [], [], []

# Set the screen dimensions
screenW = 600
screenH = 600

# Set up initial value for the timers
data_path = r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\predicted_traffic_times.xlsx'  # Path to your Excel file

while True:
    # Initialize pygame
    pygame.init()
    screen = pygame.display.set_mode((screenW, screenH))

    # Load car image
    car_image = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\cars\Down.png')  # Replace "car_down.png" with the actual path to your car image

    # Load traffic signal images
    red_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\red.png')  # Replace "red_signal.png" with the path to the red signal image
    green_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\green.png')  # Replace "green_signal.png" with the path to the green signal image
    yellow_signal = pygame.image.load(r'C:\Users\Kobiz\Documents\Term_2\AI_Tech\Project\SmartTrafficAI\img_new\Signal\yellow.png')  # Replace "yellow_signal.png" with the path to the yellow signal image

    # Traffic signal coordinates
    signalCoords = [(161, 106), (404, 106), (161, 404), (404, 404)]

    # Create a font for text rendering
    font = pygame.font.Font(None, 24)
    
    # Read timer values from Excel
    

    for index in range(len(timer_1)):  # Iterate over rows of timer values
        timer_1, timer_2, timer_3, timer_4 = read_timers_from_excel(data_path)
        # Set up initial position for the car
        car_x = 300
        car_y = 300

        # Main loop
        running = True
        current_timer = 0  # Index of the current timer being active
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            # Clear the screen
            screen.fill((255, 255, 255))

            # Blit the car onto the screen
            screen.blit(car_image, (car_x, car_y))

            # Update and render timers
            timers = [timer_1, timer_2, timer_3, timer_4]
            for i, timer in enumerate(timers):
                if timer is not None:  # Check if the timer is not None (i.e., it's not missing)
                    if i == current_timer:  # Check if this is the current active timer
                        timer_text = font.render("Timer " + str(i + 1) + ": " + str(round(timer, 2)), True, (255, 0, 0))
                    else:
                        timer_text = font.render("Timer " + str(i + 1) + ": Paused", True, (255, 0, 0))
                    if i % 2 == 0:  # Top corners
                        screen.blit(timer_text, (20, 20 + i // 2 * (screenH - 50)))
                    else:  # Bottom corners
                        screen.blit(timer_text, (screenW - 200, 20 + (i // 2) * (screenH - 50)))

                    # Decrement timer value if it is the current active timer
                    if timer > 0 and i == current_timer:
                        timers[i] -= 0.01

                    # Check if the current timer has reached 0 and update the index of the current active timer
                    if timer <= 0 and i == current_timer:
                        current_timer = (current_timer + 1) % 4

                    # Render traffic signals
                    if i == current_timer:
                        if timer > 3:  # More than 3 seconds left, display green signal
                            screen.blit(green_signal, signalCoords[i])
                        elif timer > 1:  # More than 1 second left, display yellow signal
                            screen.blit(yellow_signal, signalCoords[i])
                        else:  # Less than or equal to 1 second left, display red signal
                            screen.blit(red_signal, signalCoords[i])
                    else:
                        screen.blit(red_signal, signalCoords[i])  # Display red signal for inactive lanes

            # Update the display
            pygame.display.flip()

            # Add a small delay to control the frame rate
            pygame.time.delay(10)

        pygame.quit()
    sys.exit()


TypeError: type list doesn't define __round__ method